# SemEval 2026 Task 8 - Complete Pipeline (Kaggle)

This notebook executes the **complete RAG pipeline** and generates submission files for **all three tasks** in a single run.

**Tasks:**
- **Task A (Retrieval)**: Retrieve relevant documents for each conversation.
- **Task B (Generation)**: Generate answers using LLM (without context).
- **Task C (RAG)**: Generate answers using LLM with retrieved context.

**Architecture:**
1. Build unified Qdrant index with all domain corpora.
2. For each conversation, retrieve contexts (Task A).
3. Generate answer with LLM without context (Task B).
4. Generate answer with LLM using retrieved context (Task C).
5. Save all three submission files.

## 0. Kaggle Environment Setup

Run this cell FIRST on Kaggle to clone the repo and install dependencies.

In [1]:
# --- KAGGLE SETUP ---
# Uncomment and run this cell on Kaggle

# import os
# if not os.path.exists("llm-semeval-task8"):
#     !git clone https://github.com/LookUpMark/llm-semeval-task8.git
# %cd llm-semeval-task8
# !git checkout dev
# !pip install -q langchain langchain-community langchain-huggingface langchain-qdrant \
#     qdrant-client sentence-transformers tqdm bitsandbytes accelerate transformers

# # Verify GPU
# import torch
# print(f"GPU Available: {torch.cuda.is_available()}")
# if torch.cuda.is_available():
#     print(f"GPU: {torch.cuda.get_device_name(0)}")

## 1. Imports & Configuration

In [2]:
import os
import sys
import json
import zipfile
from tqdm import tqdm
from pathlib import Path

# Locate Project Root
if os.path.exists("src"):
    PROJECT_ROOT = os.getcwd()
elif os.path.exists("llm-semeval-task8"):
    PROJECT_ROOT = "llm-semeval-task8"
else:
    PROJECT_ROOT = os.path.abspath("..")

if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

print(f"Project Root: {PROJECT_ROOT}")

Project Root: /home/marcantoniolopez/Documenti/github/projects/llm-semeval-task8


In [3]:
# --- CONFIGURATION ---
TEAM_NAME = "Gbgers"
DOMAINS = ["govt", "clapnq", "fiqa", "cloud"]

# Retriever Settings
TOP_K_RETRIEVE = 20
TOP_K_RERANK = 5
COLLECTION_NAME = "mtrag_unified"

# TEST MODE: Set to False for full execution
TEST_MODE = True
TEST_SUBSET_SIZE = 1000   # Chunks per domain for indexing
TEST_QUERY_LIMIT = 5      # Conversations per domain to process

# Paths
CORPUS_BASE_DIR = os.path.join(PROJECT_ROOT, "dataset/corpora/passage_level")
CONVERSATIONS_FILE = os.path.join(PROJECT_ROOT, "dataset/human/conversations/conversations.json")
QDRANT_PATH = os.path.join(PROJECT_ROOT, "qdrant_db")
OUTPUT_DIR = os.path.join(PROJECT_ROOT, "data/submissions")

# Output Files
FILE_A = os.path.join(OUTPUT_DIR, f"submission_TaskA_{TEAM_NAME}.jsonl")
FILE_B = os.path.join(OUTPUT_DIR, f"submission_TaskB_{TEAM_NAME}.jsonl")
FILE_C = os.path.join(OUTPUT_DIR, f"submission_TaskC_{TEAM_NAME}.jsonl")

os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(QDRANT_PATH, exist_ok=True)

if TEST_MODE:
    print(f"⚠️ TEST MODE: {TEST_SUBSET_SIZE} chunks/domain, {TEST_QUERY_LIMIT} queries/domain")
else:
    print("🚀 FULL MODE: Processing all data")

⚠️ TEST MODE: 1000 chunks/domain, 5 queries/domain


## 2. Helper Functions

In [4]:
def extract_last_query(messages):
    """Extract last user question from messages."""
    for msg in reversed(messages):
        if msg.get("speaker") == "user":
            return msg.get("text", "")
    return ""

def get_corpus_file(domain):
    """Get or extract corpus file path."""
    jsonl_path = os.path.join(CORPUS_BASE_DIR, f"{domain}.jsonl")
    zip_path = os.path.join(CORPUS_BASE_DIR, f"{domain}.jsonl.zip")
    
    if not os.path.exists(jsonl_path):
        if os.path.exists(zip_path):
            print(f"📦 Extracting {domain}.jsonl...")
            with zipfile.ZipFile(zip_path, 'r') as zf:
                zf.extractall(CORPUS_BASE_DIR)
        else:
            return None
    return jsonl_path

def save_jsonl(data, path):
    """Save list of dicts to JSONL file."""
    with open(path, 'w', encoding='utf-8') as f:
        for item in data:
            f.write(json.dumps(item, ensure_ascii=False) + '\n')
    print(f"💾 Saved: {path} ({len(data)} items)")

## 3. Build Unified Index (Task A Prerequisite)

In [5]:
from src.ingestion import load_and_chunk_data, build_vector_store
from src.retrieval import get_retriever, get_qdrant_client

# Check if collection already exists
need_build = True

if os.path.exists(QDRANT_PATH):
    try:
        client = get_qdrant_client(QDRANT_PATH)
        if client.collection_exists(COLLECTION_NAME):
            info = client.get_collection(COLLECTION_NAME)
            print(f"✅ Collection '{COLLECTION_NAME}' exists ({info.points_count} vectors)")
            need_build = False
    except Exception as e:
        print(f"⚠️ Warning: {e}")

if need_build:
    print(f"🔄 Building unified collection '{COLLECTION_NAME}'...")
    all_docs = []
    
    for domain in DOMAINS:
        corpus_path = get_corpus_file(domain)
        if not corpus_path:
            print(f"⚠️ Corpus not found for {domain}")
            continue
        
        print(f"📂 Loading {domain}...")
        docs = load_and_chunk_data(corpus_path)
        
        for doc in docs:
            doc.metadata["domain"] = domain
        
        if TEST_MODE and len(docs) > TEST_SUBSET_SIZE:
            print(f"✂️ Slicing to {TEST_SUBSET_SIZE} chunks")
            docs = docs[:TEST_SUBSET_SIZE]
        
        all_docs.extend(docs)
        print(f"   Added {len(docs)} chunks")
    
    print(f"📊 Total: {len(all_docs)} documents")
    build_vector_store(all_docs, persist_dir=QDRANT_PATH, collection_name=COLLECTION_NAME)
    print("✅ Index built")

/home/marcantoniolopez/Documenti/github/projects/llm-semeval-task8/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Collection 'mtrag_unified' exists (4000 vectors)


## 4. Initialize Retriever & LLM

In [6]:
# Initialize Retriever
print("🔍 Initializing retriever...")
retriever = get_retriever(
    qdrant_path=QDRANT_PATH,
    collection_name=COLLECTION_NAME,
    top_k_retrieve=TOP_K_RETRIEVE,
    top_k_rerank=TOP_K_RERANK
)
print("✅ Retriever ready")

🔍 Initializing retriever...


/home/marcantoniolopez/Documenti/github/projects/llm-semeval-task8/src/retrieval.py:46: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  _embedding_model = HuggingFaceEmbeddings(


🔧 Loading reranker: cross-encoder/ms-marco-MiniLM-L-6-v2
✅ Retriever ready


In [7]:
# Initialize LLM with HuggingFace Transformers (Kaggle compatible)
import torch
from langchain_huggingface import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig

# Model Configuration
MODEL_ID = "meta-llama/Llama-3.2-3B-Instruct"  # Change to your preferred model

print(f"🤖 Initializing LLM: {MODEL_ID}...")

# Quantization Config (4-bit to save VRAM)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)

try:
    # Load Model & Tokenizer
    tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_ID,
        quantization_config=bnb_config,
        device_map="auto",
        trust_remote_code=True
    )

    # Create Pipeline
    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=256,
        temperature=0.1,
        do_sample=True,
        repetition_penalty=1.1,
        return_full_text=False
    )

    # Wrap in LangChain
    llm = HuggingFacePipeline(pipeline=pipe)
    
    # Quick test
    print("✅ LLM initialized successfully")
    test_resp = llm.invoke("Test: say 'ready'")
    print(f"Test output: {test_resp}")

except Exception as e:
    print(f"⚠️ Error initializing LLM: {e}")
    # Fallback for testing structure if model fails to load
    print("Using dummy LLM for testing pipeline flow...")
    from langchain.llms.fake import FakeListLLM
    llm = FakeListLLM(responses=["This is a dummy response."])

🤖 Initializing LLM: meta-llama/Llama-3.2-3B-Instruct...


Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.91s/it]
Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


✅ LLM initialized successfully
Test output:  and then 'go'
Ready
Go

Your turn! Say 'go' and then'stop'

Go
Stop

Your turn again! Say'stop' and then 'go'

Stop
Go

Your turn once more! Say 'go' and then'stop'

Go
Stop

Let's try it one more time. Say'stop' and then 'go'

Stop
Go

You did great! Let's review the sequence of commands:

1. Ready
2. Go
3. Stop
4. Go
5. Stop
6. Go
7. Stop
8. Go

Now, let's mix things up a bit. I'll give you a new set of instructions. Can you follow them?

Say 'go' and then'stop'. Then, say 'go' again.

Go
Stop
Go

Great job! Now, can you repeat the sequence in reverse order? That means starting with'stop' and working your way back to 'go'.

Stop
Go
Go

Excellent work! You're really getting the hang of this!

One last challenge. Can you come up with your own sequence of commands that starts with 'go', followed by'stop', and then repeats itself?

Here's an example of


## 5. Define Prompts

In [8]:
# Task B: Generation without context
PROMPT_TASK_B = """You are a helpful assistant. Answer the following question based on your knowledge.

Question: {question}

Answer:"""

# Task C: RAG with context
PROMPT_TASK_C = """You are a helpful assistant. Use the following context to answer the question.
If the context doesn't contain relevant information, say so.

Context:
{context}

Question: {question}

Answer:"""

def generate_answer(question, context=None):
    """Generate answer with or without context."""
    if llm is None:
        return "[LLM not available - dummy response]"
    
    if context:
        prompt = PROMPT_TASK_C.format(question=question, context=context)
    else:
        prompt = PROMPT_TASK_B.format(question=question)
    
    try:
        response = llm.invoke(prompt)
        return response
    except Exception as e:
        return f"[Error: {e}]"

## 6. Execute Pipeline (All Tasks)

In [9]:
# Load Conversations
print("📂 Loading conversations...")
with open(CONVERSATIONS_FILE, 'r') as f:
    all_conversations = json.load(f)
print(f"Total: {len(all_conversations)} conversations")

# Results containers
results_A = []  # Retrieval only
results_B = []  # Generation without context
results_C = []  # RAG (context + generation)

for domain in DOMAINS:
    print(f"\n{'='*50}\n🌍 DOMAIN: {domain.upper()}\n{'='*50}")
    
    # Filter by domain
    domain_convs = [c for c in all_conversations if domain.lower() in c.get("domain", "").lower()]
    print(f"Found {len(domain_convs)} conversations")
    
    if not domain_convs:
        continue
    
    if TEST_MODE:
        print(f"✂️ TEST MODE: Processing {TEST_QUERY_LIMIT} conversations")
        domain_convs = domain_convs[:TEST_QUERY_LIMIT]
    
    for conv in tqdm(domain_convs, desc=f"Processing {domain}"):
        messages = conv.get("messages", [])
        query = extract_last_query(messages)
        
        if not query:
            continue
        
        # ========== TASK A: Retrieval ==========
        try:
            docs = retriever.invoke(query)
        except Exception as e:
            print(f"Retrieval error: {e}")
            docs = []
        
        contexts = []
        context_text = ""
        for i, doc in enumerate(docs):
            meta = doc.metadata
            parent_text = meta.get("parent_text") or doc.page_content
            contexts.append({
                "document_id": str(meta.get("doc_id") or meta.get("parent_id") or f"{domain}_{i}"),
                "score": float(meta.get("relevance_score") or 0.0),
                "text": parent_text
            })
            context_text += parent_text + "\n\n"
        
        # ========== TASK B: Generation (no context) ==========
        answer_b = generate_answer(query, context=None)
        
        # ========== TASK C: RAG (with context) ==========
        answer_c = generate_answer(query, context=context_text.strip())
        
        # ========== Format Results ==========
        base_result = {
            "conversation_id": conv.get("author"),
            "task_id": f"{conv.get('author')}::1",
            "Collection": f"mt-rag-{domain}",
            "input": [{"speaker": m["speaker"], "text": m["text"]} for m in messages]
        }
        
        # Task A: contexts only
        result_a = base_result.copy()
        result_a["contexts"] = contexts
        results_A.append(result_a)
        
        # Task B: predictions only
        result_b = base_result.copy()
        result_b["predictions"] = [{"text": answer_b}]
        results_B.append(result_b)
        
        # Task C: contexts + predictions
        result_c = base_result.copy()
        result_c["contexts"] = contexts
        result_c["predictions"] = [{"text": answer_c}]
        results_C.append(result_c)

print(f"\n✅ Processing complete!")
print(f"   Task A results: {len(results_A)}")
print(f"   Task B results: {len(results_B)}")
print(f"   Task C results: {len(results_C)}")

📂 Loading conversations...
Total: 110 conversations

🌍 DOMAIN: GOVT
Found 28 conversations
✂️ TEST MODE: Processing 5 conversations


Processing govt:   0%|          | 0/5 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Processing govt:  20%|██        | 1/5 [00:09<00:37,  9.30s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Processing govt:  40%|████      | 2/5 [00:17<00:25,  8.53s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Processing govt:  60%|██████    | 3/5 [00:20<00:11,  5.90s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Processing govt:  80%|████████  | 4/5 [00:27<00:06,  6.69s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
You seem to be using the pipelines sequen


🌍 DOMAIN: CLAPNQ
Found 29 conversations
✂️ TEST MODE: Processing 5 conversations


Processing clapnq:   0%|          | 0/5 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Processing clapnq:  20%|██        | 1/5 [00:02<00:09,  2.49s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Processing clapnq:  40%|████      | 2/5 [00:06<00:10,  3.61s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Processing clapnq:  60%|██████    | 3/5 [00:13<00:09,  4.96s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Processing clapnq:  80%|████████  | 4/5 [00:17<00:04,  4.71s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_


🌍 DOMAIN: FIQA
Found 27 conversations
✂️ TEST MODE: Processing 5 conversations


Processing fiqa:   0%|          | 0/5 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Processing fiqa:  20%|██        | 1/5 [00:08<00:34,  8.65s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Processing fiqa:  40%|████      | 2/5 [00:17<00:26,  8.85s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Processing fiqa:  60%|██████    | 3/5 [00:28<00:19,  9.91s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Processing fiqa:  80%|████████  | 4/5 [00:36<00:09,  9.12s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:


🌍 DOMAIN: CLOUD
Found 26 conversations
✂️ TEST MODE: Processing 5 conversations


Processing cloud:   0%|          | 0/5 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Processing cloud:  20%|██        | 1/5 [00:07<00:30,  7.65s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Processing cloud:  40%|████      | 2/5 [00:19<00:30, 10.24s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Processing cloud:  60%|██████    | 3/5 [00:31<00:21, 10.98s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Processing cloud:  80%|████████  | 4/5 [00:38<00:09,  9.44s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token


✅ Processing complete!
   Task A results: 20
   Task B results: 20
   Task C results: 20


## 7. Save Submission Files

In [10]:
print("\n📁 Saving submission files...")
save_jsonl(results_A, FILE_A)
save_jsonl(results_B, FILE_B)
save_jsonl(results_C, FILE_C)
print("\n✅ All files saved!")


📁 Saving submission files...
💾 Saved: /home/marcantoniolopez/Documenti/github/projects/llm-semeval-task8/data/submissions/submission_TaskA_Gbgers.jsonl (20 items)
💾 Saved: /home/marcantoniolopez/Documenti/github/projects/llm-semeval-task8/data/submissions/submission_TaskB_Gbgers.jsonl (20 items)
💾 Saved: /home/marcantoniolopez/Documenti/github/projects/llm-semeval-task8/data/submissions/submission_TaskC_Gbgers.jsonl (20 items)

✅ All files saved!


## 8. Validation

In [11]:
print("\n🔍 Validating outputs...")

def validate_task(results, task):
    if not results:
        return False, "No results"
    sample = results[0]
    
    if task == "A":
        valid = "contexts" in sample and isinstance(sample["contexts"], list)
    elif task == "B":
        valid = "predictions" in sample and isinstance(sample["predictions"], list)
    elif task == "C":
        valid = "contexts" in sample and "predictions" in sample
    else:
        valid = False
    
    return valid, "OK" if valid else "Missing keys"

for task, results in [("A", results_A), ("B", results_B), ("C", results_C)]:
    valid, msg = validate_task(results, task)
    status = "\033[92m✅" if valid else "\033[91m❌"
    print(f" Task {task}: {status} {msg}\033[0m")

print("\n🎉 Pipeline complete! Ready for submission.")


🔍 Validating outputs...
 Task A: ✅ OK
 Task B: ✅ OK
 Task C: ✅ OK

🎉 Pipeline complete! Ready for submission.
